In [1]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import math

In [2]:
def get_sim_item(df_, user_col, item_col, use_iif=False):

    df = df_.copy()
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))

    user_time_ = df.groupby(user_col)['time'].agg(list).reset_index() # 引入时间因素
    user_time_dict = dict(zip(user_time_[user_col], user_time_['time']))
      
    #    商品被那些用户点击过
    item_user_ = df.groupby(item_col)[user_col].agg(set).reset_index()
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))
    
    sim_item = {}  
    item_cnt = defaultdict(int)  # 商品被点击次数
    for user, items in tqdm(user_item_dict.items()):  
        for loc1, item in enumerate(items):  
            item_cnt[item] += 1  
            sim_item.setdefault(item, {})  
            for loc2, relate_item in enumerate(items):  
                if item == relate_item:  
                    continue  
                t1 = user_time_dict[user][loc1] # 点击时间提取
                t2 = user_time_dict[user][loc2]
                 
                tmp_len = len(item_user_dict[relate_item])
                
                me = abs(len(list(item_user_dict[item]) + list(item_user_dict[relate_item])) - len(set(list(item_user_dict[item]) + list(item_user_dict[relate_item]))))
                sim_item[item].setdefault(relate_item, 0) 
                if not use_iif:  
                    if loc1-loc2>0:
                        sim_item[item][relate_item] += (0.8**(loc1-loc2-1)) * (1/((t1 - t2) * 10000+1)) + 1/math.log(1 + len(items))  # 逆向
                        #sim_item[item][relate_item] += (0.7**(loc1-loc2-1)) * (1 - (t1 - t2) * 10000) + 1 / (math.log(len(items)+1) * math.log(tmp_len+1)) # 逆向
                    else:
                        sim_item[item][relate_item] += (0.8**(loc2-loc1-1)) * (1/((t2 - t1) * 10000+1)) + 1/math.log(1 + len(items))  # 正向
                        #sim_item[item][relate_item] += (0.7**(loc2-loc1-1)) * (1 - (t2 - t1) * 10000) + 1 / (math.log(len(items)+1) * math.log(tmp_len+1)) # 正向
                else:  
                                        
                    sim_item[item][relate_item] += 1 / math.log(1 + len(items))
                    

    sim_item_corr = sim_item.copy() # 引入AB的各种被点击次数  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]) ** 0.2)

    return sim_item_corr, user_item_dict,item_user_dict

In [3]:
#def get_sim_item(df, user_col, item_col, use_iif=False):
#    #    用户点击过的商品
#    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()
#    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))
#    
#    #    商品被那些用户点击过
#    item_user_ = df.groupby(item_col)[user_col].agg(set).reset_index()
#    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))
#    sim_item = {}
#    item_cnt = defaultdict(int)
#    for user, items in tqdm(user_item_dict.items()):
#        for i in items:
#            item_cnt[i] += 1
#            sim_item.setdefault(i, {})
#            for relate_item in items:
#                if i == relate_item:
#                    continue
#                sim_item[i].setdefault(relate_item, 0)
#                if not use_iif:
#                    sim_item[i][relate_item] += 1
#                else:
#                                      
#                    me = abs(len(list(item_user_dict[i]) + list(item_user_dict[relate_item])) - len(set(list(item_user_dict[i]) + list(item_user_dict[relate_item]))))
#                    sim_item[i][relate_item] += me
#                    #sim_item[i][relate_item] += 1 / math.log(1 + len(items))
#    sim_item_corr = sim_item.copy()
#    for i, related_items in tqdm(sim_item.items()):
#        for j, cij in related_items.items():
#            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
#
#    return sim_item_corr, user_item_dict
#

In [4]:
#def recommend(sim_item_corr, user_item_dict, item_user_dict, user_id, top_k, item_num):
#
#    rank = {}  
#    interacted_items = user_item_dict[user_id] 
#    interacted_items = interacted_items[::-1]
#    for loc, i in enumerate(interacted_items):  
#        for j, wij in sorted(sim_item_corr[i].items(),key=lambda d:d[1], reverse=True):  
#            if j not in interacted_items:  
#                rank.setdefault(j, 0)  
#                #提升half
#                #rank[j] += wij * (0.7**loc) / math.log(len(item_user_dict[j]) + 1)
#                rank[j] += wij * (0.7**loc) 
#    
#    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]

In [5]:
def recommend(sim_item_corr, user_item_dict, item_user_dict,user_time_dict, user_ntime, user_id, top_k, item_num):

    rank = {}  
    interacted_items = user_item_dict[user_id] 
    interacted_items = interacted_items[::-1]
    time_seq = user_time_dict[user_id]
    time_seq = time_seq[::-1]
    for loc, i in enumerate(interacted_items):  
        t = time_seq[loc]
        for j, wij in sorted(sim_item_corr[i].items(),key=lambda d:d[1], reverse=True):  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * (0.7**loc)
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]

In [6]:
# fill user to 50 items
def get_predict(df, pred_col, top_fill):
    top_fill = [int(t) for t in top_fill.split(',')]
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]
    ids = list(df['user_id'].unique())
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])
    fill_df.sort_values('user_id', inplace=True)
    fill_df['item_id'] = top_fill * len(ids)
    fill_df[pred_col] = scores * len(ids)
    df = df.append(fill_df)
    df.sort_values(pred_col, ascending=False, inplace=True)
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)
    df = df[df['rank'] <= 50]
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',',
                                                                                                   expand=True).reset_index()
    return df

In [7]:
now_phase = 1
train_path = 'underexpose_train'
test_path = 'underexpose_test'


whole_click = pd.DataFrame()
pre2qclick = pd.DataFrame()
for c in range(now_phase + 6):
    print('phase:', c)
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,
                             names=['user_id', 'item_id', 'time'])
    qclick = pd.read_csv(test_path + '/underexpose_test_qtime-{}.csv'.format(c), header=None, names=['user_id','qtime'])
    pre2qclick = pre2qclick.append(qclick)

    all_click = click_train.append(click_test)
    
    whole_click = whole_click.append(all_click)



phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5
phase: 6


In [8]:
#    去重
whole_click = whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='first')
#    按照时间排序
whole_click = whole_click.sort_values('time')

In [9]:
item_sim_list, user_item,item_user_dict = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=False)

100%|██████████████████████████████████████████████████████████████████████████| 98769/98769 [00:35<00:00, 2750.40it/s]


In [10]:
whole_click.shape

(966669, 3)

In [11]:
k = 0
for n in tqdm(pre2qclick.values):
    a = whole_click[whole_click['user_id']==n[0]]
    if k == 0:
        new_test_click = a[a['time']<n[1]]
    else:
        new_test_click = new_test_click.append(a[a['time']<n[1]])
    k = 1

100%|███████████████████████████████████████████████████████████████████████████| 12081/12081 [01:57<00:00, 102.92it/s]


In [12]:
user_item_ = new_test_click.groupby('user_id')['item_id'].agg(list).reset_index()
user_item_dict = dict(zip(user_item_['user_id'], user_item_['item_id']))

In [13]:
user_time_ = new_test_click.groupby('user_id')['time'].agg(list).reset_index() # 引入时间因素
user_time_dict = dict(zip(user_time_['user_id'], user_time_['time']))

In [14]:
recom_item = []
k = 1
for i in tqdm(pre2qclick['user_id'].unique()):
    rank_item = recommend(item_sim_list, user_item_dict,item_user_dict,user_time_dict,float(pre2qclick[pre2qclick['user_id']==i]['qtime'].values), i, 500, 50)
    for j in rank_item:
        recom_item.append([i, j[0], j[1]])
    if k == 1:
        pass

100%|████████████████████████████████████████████████████████████████████████████| 12081/12081 [22:56<00:00, 20.87it/s]


In [15]:
top50_click = whole_click['item_id'].value_counts().index[:50].values
top50_click = ','.join([str(i) for i in top50_click])

recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])
result = get_predict(recom_df, 'sim', top50_click)
result.to_csv('baseline.csv', index=False, header=None)

In [16]:
result

,user_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,1,86677,16734,28869,32360,42199,4033,65089,30536,40660,...,30677,16491,31684,46997,69132,60466,62001,97148,25042,116065
1,2,55815,82469,92469,18104,91307,67872,67425,4612,87140,...,117360,10335,91164,83147,52254,101513,112873,99581,43876,47855
2,3,110798,28943,107129,17287,87420,69717,73088,92379,47622,...,37781,7051,24923,78914,4798,25177,39653,115266,22083,44470
3,11,26711,21517,40801,77847,6969,41221,3506,10528,59376,...,41495,29068,45634,94192,64404,104252,92755,76676,68750,104508
4,13,23554,21082,29815,41282,26131,2659,26274,37640,86964,...,106688,114792,22183,94656,8193,38524,103046,40246,48498,15026
5,15,95023,72425,7838,66393,102743,103045,54598,73953,37998,...,12203,10188,35565,66524,3693,12389,111439,72390,10443,106536
6,16,73359,91283,45966,43684,86218,22364,69207,14319,50387,...,32633,52266,87200,79596,39281,104969,36855,87519,86755,80794
7,17,107085,104978,107013,107420,108239,88288,97466,108192,83644,...,86019,106821,19223,99583,12004,69344,13110,66657,37775,53064
8,22,97356,101900,63428,98489,100827,108852,80765,110756,109803,...,107584,88169,108888,105142,14984,10513,103126,100814,97366,60547
9,25,28715,21918,10230,69478,101999,9810,33633,8245,12767,...,27383,14425,38845,89441,35863,83503,13249,9634,16427,92109
